In [1]:
import sys
sys.path.insert(1, '/kaggle/input/du2project-code/project')

In [30]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
import torch.utils.data as data
import os
from os.path import join
import argparse
import logging
from tqdm import tqdm
from torchvision.utils import save_image
#user import
from data_generator.DataLoader_Pretrain_Alexnet import CACD
from model.GAN import IPCGANs
from utils.io import check_dir,Img_to_zero_center,Reverse_zero_center
from datetime import datetime

In [2]:
TIMESTAMP = "{0:%Y-%m-%d_%H-%M-%S}".format(datetime.now())

In [15]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    
    
args = {
    'learning_rate' : 1e-4,
    'batch_size' : 256,
    'max_epoches' : 200,
    'val_interval' : 20000,
    'save_interval' : 20000, 
    'cuda_device' : '0',
    'checkpoint' : '/kaggle/working/checkpoint/pretrain_alexnet',
    'saved_model_folder' : '/kaggle/working/checkpoint/pretrain_alexnet/saved_parameters',
    'list_root' : '/kaggle/input/du2project-code/project/data/cacd2000-lists',
    'data_root' : '/kaggle/input/du2project-data/CACD2000',
    
}

args_local = {
    'learning_rate' : 1e-4,
    'batch_size' : 16,
    'max_epoches' : 200,
    'val_interval' : 20000,
    'save_interval' : 20000, 
    'cuda_device' : '0',
    'checkpoint' : './checkpoint/pretrain_alexnet/',
    'saved_model_folder' : './checkpoint/pretrain_alexnet/saved_parameters',
    'list_root' : './data/cacd2000-lists',
    'data_root' : './CACD2000',
    
}

args = dotdict(args_local)

In [7]:
os.makedirs(args.checkpoint)
os.makedirs(args.saved_model_folder)

In [16]:
from model.faceAlexnet import AgeClassify
model=AgeClassify()
optim=model.optim

In [35]:
transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((227, 227)),
        torchvision.transforms.ToTensor(),
        Img_to_zero_center()
    ])
#step4: define train/test dataloader
train_dataset = CACD(
    list_root = args.list_root, 
    data_root= args.data_root, 
    split = "train",
    transforms=transforms, label_transforms=None
)
test_dataset = CACD(
    list_root = args.list_root, 
    data_root= args.data_root, 
    split = "test",
    transforms=transforms, label_transforms=None
)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=args.batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=args.batch_size,
    shuffle=False
)

In [36]:
for epoch in range(args.max_epoches):
    pbar = tqdm(enumerate(train_loader),total = len(train_loader))
    for train_idx, (img,label) in pbar:
        img=img.cuda()
        label=label.type(torch.LongTensor)
        label=label.cuda()

        #train
        optim.zero_grad()
        model.train(img,label)
        loss=model.loss
        loss.backward()
        optim.step()
        format_str = ('step %d/%d, cls_loss = %.3f')
        pbar.set_postfix({'cls_loss': '%.3f' %(loss)})


        # save the parameters at the end of each save interval
        if train_idx*args.batch_size % args.save_interval == 0 and train_idx != 0:
            model.save_model(dir=args.saved_model_folder,
                             filename='epoch_%d_iter_%d.pth'%(epoch, train_idx))
            print('checkpoint has been created!')

        #val step

        if train_idx % args.val_interval == 0 and train_idx != 0:
            train_correct=0
            train_total=0
            with torch.no_grad():
                for val_img,val_label in tqdm(test_loader):
                    val_img=val_img.cuda()
                    val_label=val_label.cuda()
                    output=model.val(val_img)
                    train_correct += (output == val_label).sum()
                    train_total += val_img.size()[0]

            print('validate has been finished!')
            format_str = ('val_acc = %.3f')
            print(format_str % (train_correct.cpu().numpy()/train_total))

  0%|          | 5/9184 [00:01<54:46,  2.79it/s, cls_loss=1.557]  


FileNotFoundError: [Errno 2] No such file or directory: './CACD2000\\36_Penélope_Cruz_0015.jpg'